In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.utils import np_utils
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:

#building identity_block
def identity_block(X, f, filters, stage, block):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  F1, F2, F3 = filters
  X_skip = X
  #main Path
  X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform())(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), padding = "same", name = conv_name_base + '2b', kernel_initializer = glorot_uniform())(X)
  X = BatchNormalization(axis= 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding = 'same', name = conv_name_base + '2c', kernel_initializer = glorot_uniform())(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  #skip connections
  X = Add()([X_skip, X])
  X = Activation('relu')(X)

  return X

In [ ]:
#building convolution_block
def convolutional_block(X, f, filters, stage, block, s = 2):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  F1, F2, F3 = filters

  #skip
  X_skip = X

  X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  X = Conv2D(F2, (f, f), strides = (1,1), name = conv_name_base + '2b', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  #skip connections
  X_skip = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X_skip)
  X_skip = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_skip)

  X = Add()([X_skip, X])
  X = Activation('relu')(X)

  return X

In [ ]:
#baseline model - resnet50
#idea - Implementation of the ResNet50 the following architecture:
#   CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
#   -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
def ResNet50(input_shape, classes):
  X_input = Input(input_shape)
  #zero_padding
  X = ZeroPadding2D((3,3))(X_input)

  #stage1
  X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3, 3), strides=(2, 2))(X)
  #stage2
  X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
  X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
  X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
  #stage3
  X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
  #stage4
  X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
  #stage5
  X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
  #Average_pool
  X = AveragePooling2D((2, 2),padding = 'same', name = "avg_pool")(X)
  #output layers - flatten dense
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
  #creation
  model = Model(inputs = X_input, outputs = X, name='ResNet50')
  
  return model

In [ ]:
model = ResNet50(input_shape = (28, 28, 1), classes = 10)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
from keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
#preprocessing
x_train = x_train.astype('float32')/ 255.0
y_train = y_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
y_test = y_test.astype('float32')/25.0

In [ ]:
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

In [ ]:
y_train.shape

(60000,)

In [ ]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
y_train.shape

(60000, 10)

In [ ]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 34, 34, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 14, 14, 64)   3200        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 14, 14, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
model.fit(x_train, y_train, epochs = 2, batch_size = 128, shuffle=True,)

Epoch 1/2
469/469 [==============================] - 3564s 8s/step - loss: 0.0123 - accuracy: 0.9969
Epoch 2/2
469/469 [==============================] - 3573s 8s/step - loss: 1.3908e-11 - accuracy: 1.0000
